In [1]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.text_splitter import CharacterTextSplitter
import tiktoken
import os
import PyPDF2
import warnings
warnings.filterwarnings('ignore')

In [2]:
class Document:
    def __init__(self, text):
        self.page_content = text
        self.metadata = {} 

class PDFTextLoader:
    def __init__(self, file_path):
        self.file_path = file_path

    def load(self):
        documents = []
        with open(self.file_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text = page.extract_text()
                if text:
                    documents.append(Document(text))
        return documents

txt_file_path = 'file1.pdf'
loader = PDFTextLoader(file_path=txt_file_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(documents)

In [3]:
os.environ["OPENAI_API_KEY"] = "Your OPENAI_API_KEY"

In [4]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embedding=embeddings)
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(), memory=memory)

/scratch/user/fatemehdoudi/.conda/envs/in-context-learning/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/scratch/user/fatemehdoudi/.conda/envs/in-context-learning/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_open

In [5]:
query = "Summerize this document!"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/scratch/user/fatemehdoudi/.conda/envs/in-context-learning/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'This document is a report from the United States Department of Energy, titled "2020 Smart Grid System Report", published in January 2022. The report provides an in-depth analysis of the smart grid system, factors shaping its deployment, investments, technology applications, and the challenges it faces. It presents a comprehensive table of contents, which includes legislative mandates, evolution to digital sensing and control, technology availability, federal and state policies, new participants and evolving business models, resilience, implications for grid modernization, and more. It also addresses issues like coordination, integrated planning, energy justice, research, development, demonstration needs, interoperability, cybersecurity, and workforce. The document also contains references to various other reports and resources related to the topic. The report\'s intended recipients are specific members of Congress, as mentioned in the introductory message from the Secretary.'

In [6]:
query = "Summerize this document from the perspective of a power system professional."
result = conversation_chain({"question": query})
answer = result["answer"]
print(answer)

This Department of Energy document is a Smart Grid System Report from 2020. It provides a comprehensive view of the evolving electricity grid, from its initial stages of centralized generation and distribution, to its current status as an ultra-large-scale (ULS) system with decentralized data, development, and control. 

The report underlines the complexity inherent in the electric grid due to increasing consumer interaction and the influx of technology providers offering grid services traditionally supplied by utilities. It emphasizes the need for advanced technological solutions and informed decision-making to modernize the grid and meet future demands. 

The report also discusses the challenges associated with integrating renewable generation and distributed energy resources, such as wind and solar power, into the grid. These resources, which are asynchronously connected to the grid, pose new opportunities and challenges in terms of grid control and response to abnormal grid conditi